In [ ]:
import sys

sys.path.append("../..")
from typing import List, Union

from pydantic import BaseModel, Field
from rich import print

from brain.sdk import BrainClient

In [ ]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [ ]:
class OperationalResult(BaseModel):
    answer: str = Field(..., description="The answer to the input query")
    reasoning: str = Field(..., description="The reasoning behind the answer")

class RuleModification(BaseModel):
    rule_number: int = Field(..., description="The rule number to modify")
    new_content: str = Field(..., description="The new content for the rule")

class FeedbackResult(BaseModel):
    feedback: str = Field(..., description="Feedback on the previous answer")
    keep_rules: List[int] = Field(..., description="Rule numbers to keep")
    modify_rules: List[RuleModification] = Field(default_factory=list, description="Rules to modify")
    new_rules: List[str] = Field(default_factory=list, description="New rules to add")


In [ ]:
import json
from datetime import datetime
from pathlib import Path
from typing import Dict


class Rule:
    def __init__(self, content: str, rule_id: int):
        self.content = content
        self.rule_id = rule_id
        self.created_at = datetime.now()
        self.last_updated = datetime.now()

class Memory:
    def __init__(self, file_path: str = "memory.json"):
        self.file_path = Path(file_path)
        self.rules: Dict[int, Rule] = {}
        self.next_rule_id = 1
        self._load()

    def _load(self):
        if not self.file_path.exists():
            self._save()
            return

        data = json.loads(self.file_path.read_text())
        self.next_rule_id = data['next_rule_id']
        self.rules = {
            int(rule_id): Rule(
                content=rule['content'],
                rule_id=int(rule_id)
            )
            for rule_id, rule in data['rules'].items()
        }

    def _save(self):
        data = {
            'next_rule_id': self.next_rule_id,
            'rules': {
                str(rule_id): {
                    'content': rule.content,
                }
                for rule_id, rule in self.rules.items()
            }
        }
        self.file_path.write_text(json.dumps(data, indent=2))

    def get_rules_context(self) -> str:
        # if not self.rules:
        #     return "No existing rules."
        
        return "Current rules:\n" + "\n".join(
            f"{rule.rule_id}. {rule.content}"
            for rule in sorted(self.rules.values(), key=lambda x: x.rule_id)
        )

    def update_from_feedback(self, feedback: FeedbackResult):
        # Convert current rules to set for easier processing
        current_rule_ids = set(self.rules.keys())
        keep_rules = set(feedback.keep_rules)
        
        # Remove rules not in keep_rules
        rules_to_remove = current_rule_ids - keep_rules
        if rules_to_remove:
            for rule_id in rules_to_remove:
                self.rules.pop(rule_id, None)

        # Modify existing rules
        if feedback.modify_rules:
            for modification in feedback.modify_rules:
                if modification.rule_number in self.rules:
                    self.rules[modification.rule_number].content = modification.new_content
                    self.rules[modification.rule_number].last_updated = datetime.now()

        # Add new rules
        if feedback.new_rules:
            for content in feedback.new_rules:
                rule = Rule(content=content, rule_id=self.next_rule_id)
                self.rules[self.next_rule_id] = rule
                self.next_rule_id += 1

        self._save()

In [ ]:
from typing import Optional, Tuple


@brain_client.reasoner(schema=OperationalResult)
def operational_reasoner(input_query: str, context: str):
    system_prompt = """You are an agent designed to provide answers based on learned rules.
Apply the rules wisely - not every rule needs to be used for every answer.
You are an agent that uses knowledge to generate accurate, generalized answers. 
Focus on principles and patterns, not specific memorization. Use the context provided to:
1. Identify underlying patterns.
2. Apply generalized knowledge to new inputs.
Reason out the answer based on the rules you have learned.
"""

    user_prompt = f"""Query: {input_query}

{context}

Using these rules as guidelines, provide your answer:"""

    return system_prompt, user_prompt

In [ ]:
@brain_client.reasoner(schema=FeedbackResult)
def feedback_reasoner(main_goal: str, input_query: str, answer: OperationalResult, correct_answer: str, context: str):
    system_prompt = f"""You are a learning feedback agent focused on extracting deep insights aligned with a specific goal.

MAIN GOAL: {main_goal}

Your purpose is to help the system learn HOW to achieve this goal by:
1. Analyzing differences between given answers and correct answers
2. Extracting detailed knowledge that helps achieve the main goal
3. Creating comprehensive rules that guide future responses
4. Building a knowledge base focused on the goal's requirements

Core Analysis Principles:
- Extract general patterns, not specific solutions
- Look for underlying principles that achieve the goal
- Focus on knowledge that transfers to new situations
- Identify what knowledge would help achieve better answers
- Consider what deep understanding is missing
- Think about what fundamental concepts would improve performance

Rules/Knowledge Requirements:
- Must directly relate to achieving the main goal
- Should be detailed and comprehensive
- Must be generally applicable
- Should capture underlying principles
- Must focus on transferable knowledge
- Should build on existing understanding"""

    user_prompt = f"""DEEP LEARNING ANALYSIS:

GOAL TO ACHIEVE: {main_goal}

Current Learning Instance:
Input: {input_query}
Current Output: {answer.answer}
Current Reasoning: {answer.reasoning}
Expected Output: {correct_answer}

Existing Knowledge Base:
{context}

Perform Deep Analysis:
1. What fundamental understanding would help achieve the goal here?
2. What deeper patterns reveal themselves when comparing outputs?
3. What core principles would improve goal achievement?
4. What essential knowledge is missing from our current rules?
5. What broader understanding would help with similar goals?

Provide:
1. Rich Analysis: Detailed feedback about what we can learn regarding our goal
2. Retained Knowledge: List numbers of rules that contain valid, goal-relevant principles
3. Knowledge Updates: List [[rule_number, "enhanced knowledge or principle"]]
4. New Understanding: Add detailed new rules that capture essential goal-relevant knowledge

Focus Areas:
- Deep understanding over surface patterns
- Core principles over specific solutions
- Transferable knowledge over contextual details
- Fundamental concepts over specific applications
- Goal achievement mechanisms over input-output pairs
Your goal is to help the system learn to achieve its main goal by analyzing successes and failures.
- Identify positive patterns that led to correct answers.
- Identify negative patterns or gaps that caused failures.
- Provide insights to refine knowledge for future queries.
- Focus on creating rules or principles that generalize well.


- If something is working give positive feedback so that the system can learn from it
- If something is not working give negative feedback so that the system can learn from it

"""

    return system_prompt, user_prompt

In [ ]:
memory = Memory()
operational_reasoner_id = operational_reasoner.register()
feedback_reasoner_id = feedback_reasoner.register()

In [ ]:
def process_query(
    input_query: str,
    correct_answer: Optional[str] = None,
    main_goal: str = "Learn to provide accurate and consistent responses",
    memory: Memory = None,
):
    if memory is None:
        memory = Memory()
    context = memory.get_rules_context()
    # Get answer using current rules
    result = brain_client.use(operational_reasoner_id)(
        input_query=input_query, context=context
    )

    if correct_answer is not None:
        # Get feedback and evolve rules
        feedback = brain_client.use(feedback_reasoner_id)(
            main_goal=main_goal,
            input_query=input_query,
            answer=result,
            correct_answer=correct_answer,
            context=context,
        )
        # Update memory based on feedback
        memory.update_from_feedback(feedback)

        return {"answer": result.answer, "feedback": feedback}

    return {"answer": result.answer}

In [ ]:
# query1 = "What is the capital of France?"
# result1 = process_query(
#     input_query=query1,
#     correct_answer="Paris.",
#     main_goal="Learn how to format the answer exactly in given style"
# )
# print(result1)

In [ ]:
training_examples = [
    {
        "query": "Transform 'hello'",
        "correct_answer": "Ellohay"
    },
    {
        "query": "Transform 'world'",
        "correct_answer": "Orldway"
    },
    {
        "query": "Transform 'python'",
        "correct_answer": "Ythonpay"
    },
    {
        "query": "Transform 'code'",
        "correct_answer": "Odecay"
    },
    {
        "query": "Transform 'learn'",
        "correct_answer": "Earnlay"
    }
]

def train_pattern_learner():
    memory = Memory()  # Reset memory for fresh learning
    
    for _ in range(1):
        for i, example in enumerate(training_examples, 1):
            
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation pattern that converts the input to output"
            )
            
            print(f"System Answer: {result['answer']} | Correct Answer: {example['correct_answer']}")
        print("-" * 50)

train_pattern_learner()

System Answer: ellohay | Correct Answer: Ellohay
System Answer: orldway | Correct Answer: Orldway
System Answer: ythonpay | Correct Answer: Ythonpay
System Answer: odecay | Correct Answer: Odecay
System Answer: earnlay | Correct Answer: Earnlay
--------------------------------------------------


In [ ]:
from builtins import print


def transform_sentence(sentence,n=2):
    n-=1
    words = sentence.split()
    transformed = ''.join(word[n] for word in words if len(word) >=n)
    return transformed


# Training and testing data generation
sentences = [
    "python is fun",
    "code is life",
    "learn and grow",
    "practice makes perfect",
    "knowledge is power",
    "ai is the future",
    "data drives decisions",
    "science is amazing",
    "always keep learning",
    "perseverance pays off",
    "learning is ongoing",
    "practice brings mastery",
    "knowledge fuels creativity",
    "transform ideas daily",
    "explore and create",
    "analyze your potential",
    "never stop growing",
    "think outside boundaries",
    "focus on improvement",
    "believe in progress"
]

nth_word=2
# Generate examples
examples = [
    {
        "query": f"Transform '{sentence}'",
        "correct_answer": transform_sentence(sentence,n=nth_word)
    }
    for sentence in sentences
]

# Split examples into training and testing (10 each)
training_examples = examples[:10]
testing_examples = examples[10:]


# Modified train_pattern_learner function
def train_pattern_learner():
    memory = Memory(file_path="memory_n2.json")  # Reset memory for fresh learning
    for _ in range(10):
        print("Training on one epoch...\n")
        for i, example in enumerate(training_examples, 1):
            # print(f"Training Example {i}: {example['query']}")
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation rule that converts the input to output, and return the correct output",
                memory=memory,
            )
            print(f"System Answer: {result.get('answer', 'No Answer')} | Correct Answer: {example['correct_answer']}")
        
        print("-" * 50)

train_pattern_learner()

Training on one epoch...

System Answer: pythn sfn | Correct Answer: ysu
System Answer: cre8 life | Correct Answer: osi
System Answer: evolve | Correct Answer: enr
System Answer: Prac makes perf | Correct Answer: rae
System Answer: Power of knowledge | Correct Answer: nso
System Answer: AI is the future. | Correct Answer: ishu
System Answer: data informs choices | Correct Answer: are
System Answer: science is wonderful | Correct Answer: csm
System Answer: akl | Correct Answer: lee
System Answer: prsvrncpysf | Correct Answer: eaf
--------------------------------------------------
Training on one epoch...

System Answer: ptyh | Correct Answer: ysu
System Answer: cde is lf | Correct Answer: osi
System Answer: evolve | Correct Answer: enr
System Answer: practc mks perfct | Correct Answer: rae
System Answer: knwlge pwr | Correct Answer: nso
System Answer: AI: Future | Correct Answer: ishu
System Answer: data drives decisions | Correct Answer: are
System Answer: Sci is awe | Correct Answer: 

KeyboardInterrupt: 

## Enhanced multi memory CLU

```mermaid
graph TD
    A[Input Query] --> B[Operational Reasoner]
    B --> C[Generated Answer]
    C --> D[Feedback Reasoner]
    E[Correct Answer] --> D
    F[Main Goal] --> D
    
    subgraph Memory System
        G[Pattern Memory]
        H[Success Memory]
        I[Failure Memory]
        J[Confidence Tracker]
    end
    
    G --> B
    H --> B
    I --> B
    J --> B
    
    D --> K[Memory Manager]
    K --> G
    K --> H
    K --> I
    K --> J

    subgraph Feedback Analysis
        L[Pattern Extractor]
        M[Success Analyzer]
        N[Failure Analyzer]
        O[Confidence Updater]
    end
    
    D --> L
    D --> M
    D --> N
    D --> O
```

In [101]:
import json
from datetime import datetime
from pathlib import Path
from typing import Dict, List


class MemoryEntry:
    def __init__(self, content: str, knowledge_type: str, entry_id: int):
        self.content = content  # Generalized rule/knowledge
        self.knowledge_type = knowledge_type  # 'positive', 'negative', 'general'
        self.entry_id = entry_id
        self.created_at = datetime.now()
        self.last_updated = datetime.now()


class Memory:
    def __init__(self, file_path: str = "memory.json"):
        self.file_path = Path(file_path)
        self.entries: Dict[int, MemoryEntry] = {}
        self.next_entry_id = 1
        self._load()

    def _load(self):
        if not self.file_path.exists():
            # Initialize with default values if the file does not exist
            self._save()
            return

        try:
            data = json.loads(self.file_path.read_text())
            self.next_entry_id = data.get('next_entry_id', 1)  # Default to 1 if key is missing
            self.entries = {
                int(entry_id): MemoryEntry(
                    content=entry['content'],
                    knowledge_type=entry['knowledge_type'],
                    entry_id=int(entry_id)
                )
                for entry_id, entry in data.get('entries', {}).items()  # Default to empty if 'entries' is missing
            }
        except (json.JSONDecodeError, KeyError):
            # If the file is corrupted or has unexpected structure, reinitialize
            self.next_entry_id = 1
            self.entries = {}
            self._save()


    def _save(self):
        data = {
            'next_entry_id': self.next_entry_id,
            'entries': {
                str(entry_id): {
                    'content': entry.content,
                    'knowledge_type': entry.knowledge_type
                }
                for entry_id, entry in self.entries.items()
            }
        }
        self.file_path.write_text(json.dumps(data, indent=2))

    def get_context(self) -> str:
        positive_knowledge = [
            f"{entry.entry_id}. {entry.content}"
            for entry in sorted(self.entries.values(), key=lambda x: x.entry_id)
            if entry.knowledge_type == "positive"
        ]

        general_knowledge = [
            f"{entry.entry_id}. {entry.content}"
            for entry in sorted(self.entries.values(), key=lambda x: x.entry_id)
            if entry.knowledge_type == "general"
        ]

        negative_knowledge = [
            f"{entry.entry_id}. {entry.content}"
            for entry in sorted(self.entries.values(), key=lambda x: x.entry_id)
            if entry.knowledge_type == "negative"
        ]

        context = "## Knowledge Base\n"

        if positive_knowledge:
            context += "### Positive Knowledge:\n"
            context += (
                "This knowledge reflects principles that have consistently worked well to achieve the main goal.\n"
                "Use these as a primary guide when generating responses.\n"
            )
            context += "\n".join(positive_knowledge) + "\n\n"

        if general_knowledge:
            context += "### General Knowledge:\n"
            context += (
                "This knowledge contains broader principles or patterns that can be applied to multiple scenarios.\n"
                "Use these when no specific positive knowledge applies.\n"
            )
            context += "\n".join(general_knowledge) + "\n\n"

        if negative_knowledge:
            context += "### Negative Knowledge:\n"
            context += (
                "This knowledge reflects patterns or principles that did not work well or caused errors in the past.\n"
                "Avoid using these when generating responses.\n"
            )
            context += "\n".join(negative_knowledge) + "\n\n"

        if not (positive_knowledge or general_knowledge or negative_knowledge):
            context += "No knowledge entries available.\n"

        return context


    def update_from_feedback(self, feedback_result: FeedbackResult):
        # Prune outdated knowledge
        if feedback_result.prune_entries:
            for entry_id in feedback_result.prune_entries:
                if entry_id in self.entries:
                    del self.entries[entry_id]

        # Update existing knowledge
        for update in feedback_result.update_entries:
            if update.entry_id is not None and update.entry_id in self.entries:
                self.entries[update.entry_id].content = update.content  # Use `content`
                self.entries[update.entry_id].knowledge_type = update.knowledge_type
                self.entries[update.entry_id].last_updated = datetime.now()

        # Add new knowledge
        for new_entry in feedback_result.new_entries:
            entry = MemoryEntry(
                content=new_entry.content,
                knowledge_type=new_entry.knowledge_type,
                entry_id=self.next_entry_id
            )
            self.entries[self.next_entry_id] = entry
            self.next_entry_id += 1

        self._save()

In [112]:
from typing import List, Optional

from pydantic import BaseModel, Field


class OperationalResult(BaseModel):
    answer: str = Field(..., description="Generated answer for the input query")
    explanation: str = Field(..., description="Explanation or reasoning behind the answer")


class FeedbackEntry(BaseModel):
    entry_id: Optional[int] = Field(None, description="ID of the knowledge entry being updated (if applicable)")
    content: str = Field(..., description="Generalized knowledge or pattern to store")
    knowledge_type: str = Field(..., description="Type of knowledge: 'positive', 'negative', 'general'")


class FeedbackResult(BaseModel):
    prune_entries: List[int] = Field(default_factory=list, description="IDs of knowledge to prune")
    update_entries: List[FeedbackEntry] = Field(default_factory=list, description="Knowledge entries to update")
    new_entries: List[FeedbackEntry] = Field(default_factory=list, description="New knowledge entries to add")

In [116]:
@brain_client.reasoner(schema=OperationalResult)
def operational_reasoner(input_query: str, context: str):
    system_prompt = """You are a reasoning agent tasked with applying generalized knowledge to generate accurate and goal-aligned responses."""
    user_prompt = f"""
    ## Task:
    Use the knowledge provided in the context to generate the most accurate and goal-aligned answer for the query.

    ## Query:
    {input_query}

    ## Context:
    {context}

    ### Instructions:
    - Apply the most relevant generalized knowledge from the context.
    - Avoid directly using specific input-output examples unless they represent generalizable principles.
    - Ensure your response aligns with the main goal of the system.
    - Penalize using incorrect or irrelevant knowledge while emphasizing positive and generalizable knowledge.
    """
    return system_prompt, user_prompt

@brain_client.reasoner(schema=FeedbackResult)
def feedback_reasoner(main_goal: str, input_query: str, generated_answer: OperationalResult, correct_answer: str, context: str):
    system_prompt = """You are a feedback agent focused on improving the system's ability to achieve its main goal by analyzing the interaction and refining the knowledge base."""
    user_prompt = f"""
    ## Main Goal:
    {main_goal}

    ## Query:
    {input_query}

    ## Generated Answer:
    {generated_answer.answer}

    ## Explanation of Generated Answer:
    {generated_answer.explanation}

    ## Correct Answer:
    {correct_answer}

    ## Current Knowledge Base Context:
    {context}

    ### Reinforcement Learning Approach:
    Your role is to act like a reinforcement learning feedback system. Analyze the interaction and:
    - Award points for positive contributions toward achieving the main goal.
    - Penalize points for incorrect or irrelevant contributions.
    - Extract general patterns, principles, or rules that align with the main goal.
    - Focus on ensuring the system generalizes knowledge effectively to future tasks.

    ### Tasks:

    #### 1. Identify and Reward Positive Contributions
    - Identify which parts of the generated answer align with the correct answer and the main goal.
    - Highlight the knowledge items that contributed positively and explain why they worked.
    - Mark these as **positive knowledge** with rewards to encourage their future use.

    #### 2. Penalize and Correct Negative Contributions
    - Identify which parts of the generated answer do not align with the correct answer or the main goal.
    - Highlight knowledge items that caused errors or failed to contribute meaningfully.
    - Provide an analysis of why they did not work and mark them as **negative knowledge** with penalties to discourage their use.

    #### 3. Extract Generalizable Patterns
    - Propose general patterns, transformations, or principles that can guide the system for similar queries.
    - Focus on reusable rules rather than memorized input-output pairs.
    - Add these as **general knowledge** for future guidance.

    #### 4. Suggest Updates to Knowledge
    - For existing knowledge:
      - Recommend modifications to improve alignment with the main goal or make them more generalizable.
    - For new knowledge:
      - Propose generalized rules derived from the interaction to enhance future responses.

    #### 5. Prune Knowledge
    - Identify knowledge that is irrelevant, overly specific, or redundant.
    - Suggest pruning such knowledge to maintain an efficient and effective knowledge base.

    ### Examples of Feedback:
    - If a knowledge item consistently helps achieve the main goal, award it positive points and refine it as needed.
    - If a knowledge item introduces errors, penalize it and suggest corrections or pruning.
    - If a pattern (e.g., "Move initial consonants to the end and append 'ay'") is generalizable, add it as new general knowledge.
    - make the feedback detailed and actionable to guide the system's learning process effectively.
    
    ### Provide Detailed Feedback:
    - Clearly explain the reasoning behind awarding or penalizing knowledge.
    - Ensure all suggestions are actionable and aligned with the main goal.
    - Focus on concise, generalizable insights to refine the system's ability to learn and improve.

    ### Guidelines:
    - Positive knowledge: Encourage reuse by awarding it.
    - Negative knowledge: Penalize and discourage its use.
    - General knowledge: Extract transferable principles for unseen data.
    """
    return system_prompt, user_prompt


In [117]:
#register the reasoners
operational_reasoner_id = operational_reasoner.register()
feedback_reasoner_id = feedback_reasoner.register()

def process_query(input_query: str, correct_answer: Optional[str] = None, main_goal: str = "Learn patterns and transformations",memory=None):
    if memory is None:
        memory = Memory()
    context = memory.get_context()

    # Operational Reasoner
    result = brain_client.use(operational_reasoner_id)(
        input_query=input_query,
        context=context
    )

    if correct_answer is not None:
        # Feedback Reasoner
        feedback = brain_client.use(feedback_reasoner_id)(
            main_goal=main_goal,
            input_query=input_query,
            generated_answer=result,
            correct_answer=correct_answer,
            context=context
        )
        # Update memory
        memory.update_from_feedback(feedback)

        return {"answer": result.answer, "feedback": feedback}

    return {"answer": result.answer}

In [94]:
training_examples = [
    {"query": "Transform 'hello'", "correct_answer": "Ellohay"},
    {"query": "Transform 'world'", "correct_answer": "Orldway"},
    {"query": "Transform 'python'", "correct_answer": "Ythonpay"},
    {"query": "Transform 'code'", "correct_answer": "Odecay"},
    {"query": "Transform 'learn'", "correct_answer": "Earnlay"}
]

def train_pattern_learner():
    memory = Memory(file_path="memory_n2.json")
    for _ in range(3):  # Iterate multiple times to reinforce learning
        for example in training_examples:
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn to extract and apply transformation patterns",
                memory=memory
            )
            print(f"Answer: {result['answer']} | Correct: {example['correct_answer']}")

train_pattern_learner()

Memory saved to memory_n2.json

Memory saved to memory_n2.json

Answer: olleh | Correct: Ellohay

Memory saved to memory_n2.json

Answer: orldway | Correct: Orldway

...

Answer: ythonpay | Correct: Ythonpay

Memory saved to memory_n2.json

Answer: odecay | Correct: Odecay

Memory saved to memory_n2.json

Answer: earnlay | Correct: Earnlay

In [119]:
from builtins import print


def transform_sentence(sentence,n=2):
    n-=1
    words = sentence.split()
    transformed = ''.join(word[n] for word in words if len(word) >=n)
    return transformed


# Training and testing data generation
sentences = [
    "python is fun",
    "code is life",
    "learn and grow",
    "practice makes perfect",
    "knowledge is power",
    "ai is the future",
    "data drives decisions",
    "science is amazing",
    "always keep learning",
    "perseverance pays off",
    "learning is ongoing",
    "practice brings mastery",
    "knowledge fuels creativity",
    "transform ideas daily",
    "explore and create",
    "analyze your potential",
    "never stop growing",
    "think outside boundaries",
    "focus on improvement",
    "believe in progress"
]

nth_word=2
# Generate examples
examples = [
    {
        "query": f"Transform '{sentence}'",
        "correct_answer": transform_sentence(sentence,n=nth_word)
    }
    for sentence in sentences
]

# Split examples into training and testing (10 each)
training_examples = examples[:10]
testing_examples = examples[10:]


# Modified train_pattern_learner function
def train_pattern_learner():
    memory = Memory(file_path="memory_n3.json")  # Reset memory for fresh learning
    for _ in range(3):
        print("Training on one epoch...\n")
        for i, example in enumerate(training_examples, 1):
            # print(f"Training Example {i}: {example['query']}")
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation rule that converts the input to output",
                memory=memory
            )
            print(f"System Answer: {result.get('answer', 'No Answer')} | Correct Answer: {example['correct_answer']}")
        
        print("-" * 50)

train_pattern_learner()

Training on one epoch...

System Answer: PIF (Python Is Fun) | Correct Answer: ysu
System Answer: CISL | Correct Answer: osi
System Answer: L&G | Correct Answer: enr
System Answer: PracPerf | Correct Answer: rae
System Answer: K = P | Correct Answer: nso
System Answer: AI: The Future | Correct Answer: ishu
System Answer: d2d | Correct Answer: are
System Answer: Sci-mazing! | Correct Answer: csm
System Answer: Lifelong Growth | Correct Answer: lee
System Answer: Persist for success | Correct Answer: eaf
--------------------------------------------------
Training on one epoch...

System Answer: py is fun! | Correct Answer: ysu
System Answer: C.Life | Correct Answer: osi
System Answer: L&G | Correct Answer: enr
System Answer: pract-makes perf | Correct Answer: rae
System Answer: K is P | Correct Answer: nso
System Answer: A.I. = Future | Correct Answer: ishu
System Answer: D2D (Data Drives Decisions) | Correct Answer: are
System Answer: Sci Amz | Correct Answer: csm
System Answer: Lifelon

# New test

In [129]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime
import json
from pathlib import Path


In [145]:
class OperationalResult(BaseModel):
    answer: str = Field(..., description="The answer to the input query based on current knowledge.")
    explanation: str = Field(..., description="The reasoning behind the answer.")

class RuleModification(BaseModel):
    knowledge_id: int = Field(..., description="The knowledge item ID to modify.")
    new_content: str = Field(..., description="The new content for the knowledge item.")

class FeedbackResult(BaseModel):
    feedback: str = Field(..., description="Detailed feedback on the previous answer.")
    retain_knowledge_ids: List[int] = Field(
        ..., description="IDs of knowledge items to retain as they are."
    )
    modify_knowledge: List[RuleModification] = Field(
        default_factory=list, description="List of knowledge items to modify."
    )
    add_positive_knowledge: List[str] = Field(
        default_factory=list, description="New positive knowledge items to add."
    )
    add_negative_knowledge: List[str] = Field(
        default_factory=list, description="New negative knowledge items to add."
    )
    remove_knowledge_ids: List[int] = Field(
        default_factory=list, description="IDs of knowledge items to remove."
    )

class KnowledgeItem(BaseModel):
    id: int = Field(..., description="Unique identifier for the knowledge item.")
    content: str = Field(..., description="The content of the knowledge item.")
    type: str = Field(..., description="Type of knowledge: 'positive' or 'negative'.")
    created_at: datetime = Field(default_factory=datetime.now, description="Creation timestamp.")
    last_updated: datetime = Field(default_factory=datetime.now, description="Last update timestamp.")


In [183]:
class Memory:
    def __init__(self, file_path: str = "memory.json"):
        self.file_path = Path(file_path)
        self.positive_knowledge: List[KnowledgeItem] = []
        self.negative_knowledge: List[KnowledgeItem] = []
        self.next_knowledge_id = 1
        self._load()

    def _load(self):
        if not self.file_path.exists():
            self._save()
            return

        data = json.loads(self.file_path.read_text())
        self.next_knowledge_id = data.get('next_knowledge_id', 1)
        self.positive_knowledge = [KnowledgeItem(**item) for item in data.get('positive_knowledge', [])]
        self.negative_knowledge = [KnowledgeItem(**item) for item in data.get('negative_knowledge', [])]

    def _save(self):
        data = {
            'next_knowledge_id': self.next_knowledge_id,
            'positive_knowledge': [item.model_dump() for item in self.positive_knowledge],
            'negative_knowledge': [item.model_dump() for item in self.negative_knowledge],
        }
        # Ensure datetime objects are serialized as strings
        self.file_path.write_text(json.dumps(data, default=str, indent=2))

    def get_positive_knowledge_context(self) -> str:
        if not self.positive_knowledge:
            return "No positive knowledge available."
        return "Positive Knowledge (use IDs to refer):\n" + "\n".join(
            f"{item.id}. {item.content}" for item in self.positive_knowledge
        )

    def get_negative_knowledge_context(self) -> str:
        if not self.negative_knowledge:
            return "No negative knowledge available."
        return "Negative Knowledge (use IDs to refer):\n" + "\n".join(
            f"{item.id}. {item.content}" for item in self.negative_knowledge
        )

    def update_from_feedback(self, feedback: FeedbackResult):
        # Retain specified knowledge items
        retain_ids = set(feedback.retain_knowledge_ids)
        self.positive_knowledge = [item for item in self.positive_knowledge if item.id in retain_ids]
        self.negative_knowledge = [item for item in self.negative_knowledge if item.id in retain_ids]

        # Modify existing knowledge
        if feedback.modify_knowledge:
            for mod in feedback.modify_knowledge:
                # Search in positive and negative knowledge
                for item in self.positive_knowledge + self.negative_knowledge:
                    if item.id == mod.knowledge_id:
                        item.content = mod.new_content
                        item.last_updated = datetime.now()
                        break

        # Remove knowledge items
        if feedback.remove_knowledge_ids:
            self.positive_knowledge = [item for item in self.positive_knowledge if item.id not in feedback.remove_knowledge_ids]
            self.negative_knowledge = [item for item in self.negative_knowledge if item.id not in feedback.remove_knowledge_ids]

        # Add new positive knowledge
        if feedback.add_positive_knowledge:
            for content in feedback.add_positive_knowledge:
                new_item = KnowledgeItem(
                    id=self.next_knowledge_id,
                    content=content,
                    type="positive",
                )
                self.positive_knowledge.append(new_item)
                self.next_knowledge_id += 1

        # Add new negative knowledge
        if feedback.add_negative_knowledge:
            for content in feedback.add_negative_knowledge:
                new_item = KnowledgeItem(
                    id=self.next_knowledge_id,
                    content=content,
                    type="negative",
                )
                self.negative_knowledge.append(new_item)
                self.next_knowledge_id += 1

        self._save()


In [184]:
@brain_client.reasoner(schema=OperationalResult)
def operational_reasoner(input_query: str, positive_context: str, negative_context: str):
    system_prompt = """You are an intelligent agent designed to provide accurate and consistent answers based on generalized knowledge.
Apply the positive knowledge effectively and avoid using knowledge items classified as negative.
Focus on transferring general principles to solve new queries."""

    user_prompt = f"""Query: {input_query}

{positive_context}

{negative_context}

Important:
- Use the positive knowledge items to guide your answer.
- Avoid using any strategies or patterns mentioned in the negative knowledge items.
- Do not memorize specific input-output pairs; instead, apply general principles.
- Provide your answer that aligns with the main goal."""

    return system_prompt, user_prompt

# @brain_client.reasoner(schema=FeedbackResult)
# def feedback_reasoner(main_goal: str, input_query: str, answer: OperationalResult, correct_answer: str, positive_context: str, negative_context: str):
#     system_prompt = f"""You are a sophisticated learning feedback agent focused on extracting and refining knowledge to achieve the main goal.

# MAIN GOAL: {main_goal}

# Your tasks:
# 1. Analyze the differences between the system's answer and the correct answer.
# 2. Understand ad give feeddback on the system's reasoning based on the main goal
# 2. Extract generalized, transferable knowledge that aligns with the main goal.
# 3. Identify what worked (positive knowledge) and what didn't (negative knowledge).
# 4. Suggest modifications to existing knowledge or propose new knowledge items.
# 5. Ensure that the extracted knowledge enhances the system's ability to generalize to unseen data.

# Core Analysis Principles:
# - Focus on underlying principles and patterns.
# - Separate positive insights from negative ones.
# - Ensure knowledge is goal-oriented and transferable."""

#     user_prompt = f"""DEEP LEARNING ANALYSIS:

# GOAL TO ACHIEVE: {main_goal}

# Current Learning Instance:
# Input: {input_query}
# ---
# System Output: {answer.answer}
# System Explanation: {answer.explanation}
# ----
# Expected Output: {correct_answer}

# Existing Positive Knowledge:
# {positive_context}

# Existing Negative Knowledge:
# {negative_context}

# Instructions:
# - Use the IDs provided to refer to specific knowledge items.
# - Do not memorize specific input-output pairs.
# - Provide feedback in terms of general principles and patterns.
# - Understand ad give feeddback on the system's reasoning based on the main goal

# Perform Deep Analysis:
# 1. What fundamental understanding would help achieve the goal here?
# 2. What deeper patterns are revealed when comparing outputs?
# 3. What core principles would improve goal achievement?
# 4. What essential knowledge is missing from our current positive knowledge?
# 5. What negative knowledge should be added based on what did not work?

# Provide:
# 1. Rich Analysis: Detailed feedback on what can be learned regarding the main goal.
# 2. Retain Knowledge IDs: List IDs of positive and negative knowledge items that should be retained.
# 3. Knowledge Modifications: List of modifications in the format [[knowledge_id, "enhanced or corrected content"]] for existing knowledge items to modify.
# 4. New Positive Knowledge: Add detailed new positive knowledge items that capture essential goal-relevant information.
# 5. New Negative Knowledge: Add detailed new negative knowledge items that capture what should be avoided.
# 6. Remove Knowledge IDs: List IDs of knowledge items that should be removed.
# 7. make the knowledge detailed and actionable to guide the system's learning process effectively.
# 8. You will add new knowledge items if you see that the existing knowledge is not enough to achieve the main goal for future similar (Not same) queries.
# 9. You will remove exieting knowledge if you think it is wrong to achieve the main goal. But note if you think it might be relevant for future  similar (Not same) queries, you can keep it.
# 10. You will modify existing knowledge if you think it is not enough to achieve the main goal for future  similar (Not same) queries.
# 11. You will retain existing knowledge if you think it is important to achieve the main goal for future similar (Not same) queries.

# Focus Areas:
# - Deep understanding over surface patterns.
# - Core principles over specific solutions.
# - Transferable knowledge over contextual details.
# - Fundamental concepts over specific applications.
# - Goal achievement mechanisms over input-output pairs.

# - If something is working, provide positive feedback to reinforce it.
# - If something is not working, provide negative feedback to suppress it.
# """

#     return system_prompt, user_prompt


In [185]:
@brain_client.reasoner(schema=FeedbackResult)
def feedback_reasoner(
    main_goal: str,
    input_query: str,
    answer: OperationalResult,
    correct_answer: str,
    positive_context: str,
    negative_context: str
):
    system_prompt = f"""You are a sophisticated learning feedback agent dedicated to extracting and refining knowledge to achieve the main goal.

MAIN GOAL: {main_goal}

Your Objectives:
1. Analyze the differences between the system's answer and the correct answer.
2. Provide feedback on the system's reasoning in relation to the main goal.
3. Extract generalized, transferable knowledge that aligns with the main goal.
4. Identify what worked (positive knowledge) and what didn't work (negative knowledge).
5. Suggest modifications to existing knowledge or propose new knowledge items.
6. Enhance the system's ability to generalize to unseen data through the extracted knowledge.

Core Principles:
- Focus on underlying principles and patterns.
- Separate positive insights from negative ones.
- Ensure all knowledge is goal-oriented and transferable.
"""

    user_prompt = f"""# DEEP LEARNING ANALYSIS

**GOAL TO ACHIEVE:** {main_goal}

## Current Learning Instance:
- **Input Query:** {input_query}
- **System Output:** {answer.answer}
- **System Explanation:** {answer.explanation}
- **Expected Output:** {correct_answer}

## Existing Knowledge:
### Positive Knowledge:
{positive_context}

### Negative Knowledge:
{negative_context}

## Your Tasks:

### 1. Analysis:
- Compare the system's output with the expected output.
- Analyze the system's reasoning in relation to the main goal.
- Identify fundamental understandings that would help achieve the goal.
- Reveal deeper patterns when comparing outputs.
- Determine core principles that would improve goal achievement.

### 2. Feedback:
- Provide detailed feedback on what can be learned regarding the main goal.
- Highlight what worked (positive aspects) and what didn't work (negative aspects).
- Focus on general principles and patterns, not specific input-output pairs.

### 3. Knowledge Management:
- **Retain Knowledge**: List IDs of positive and negative knowledge items that should be retained as they are.
- **Modify Knowledge**: Suggest modifications to existing knowledge items that need enhancement or correction. Provide in the format:
[[knowledge_id, "enhanced or corrected content"]]
- **Add New Knowledge**:
- **New Positive Knowledge**: Add detailed new positive knowledge items that capture essential goal-relevant information.
- **New Negative Knowledge**: Add detailed new negative knowledge items that capture what should be avoided.
- **Remove Knowledge**: List IDs of knowledge items that should be removed if they are incorrect or unhelpful toward achieving the main goal.

## Instructions:

- **Use the provided IDs** to refer to specific knowledge items.
- **Do not memorize specific input-output pairs**; focus on generalizing principles to future similar queries.
- **Make the knowledge detailed and actionable** to effectively guide the system's learning process.
- **Add new knowledge** if existing knowledge is insufficient for future similar (not identical) queries.
- **Remove existing knowledge** only if it is incorrect and not relevant for future similar queries.
- **Modify existing knowledge** if it needs enhancement to achieve the main goal for future similar queries.
- **Retain existing knowledge** if it is important for achieving the main goal in future similar queries.

## Focus Areas:

- Emphasize **deep understanding** over surface patterns.
- Prioritize **core principles** over specific solutions.
- Ensure knowledge is **transferable** to new, unseen data.
- Focus on **fundamental concepts** over specific applications.
- Concentrate on mechanisms that achieve the **main goal** over specific input-output pairs.

- If something is working, **provide positive feedback** to reinforce it.
- If something is not working, **provide negative feedback** to suppress it.

Important Notes:

Ensure that all parts of the output are filled appropriately.
Include all reasoning and analysis within the "feedback" field.
Be concise but thorough in your feedback and knowledge suggestions.
Do not include any extraneous text outside of the specified output format.
Focus on extracting general principles that will help improve future performance toward achieving the main goal.
"""
    return system_prompt, user_prompt

In [186]:
# Assuming brain_client is already defined and set up
# You may need to import or define brain_client as per your environment
# For the purpose of this code, we assume brain_client is available

operational_reasoner_id = operational_reasoner.register()
feedback_reasoner_id = feedback_reasoner.register()


In [187]:
def process_query(input_query: str, correct_answer: Optional[str] = None, 
                  main_goal: str = "Learn to provide accurate and consistent responses",
                  memory: Optional[Memory] = None):
    if memory is None:
        memory = Memory()
    positive_context = memory.get_positive_knowledge_context()
    negative_context = memory.get_negative_knowledge_context()

    # Get answer using current positive and negative knowledge
    result = brain_client.use(operational_reasoner_id)(
        input_query=input_query,
        positive_context=positive_context,
        negative_context=negative_context
    )

    if correct_answer is not None:
        # Get feedback and evolve knowledge
        feedback = brain_client.use(feedback_reasoner_id)(
            main_goal=main_goal,
            input_query=input_query,
            answer=result,
            correct_answer=correct_answer,
            positive_context=positive_context,
            negative_context=negative_context
        )
        # Update memory based on feedback
        memory.update_from_feedback(feedback)
        # We let the LLM decide what knowledge to remove or keep
        return {"answer": result.answer, "feedback": feedback}
    
    return {"answer": result.answer}


In [188]:
training_examples = [
    {
        "query": "Transform 'hello'",
        "correct_answer": "Ellohay"
    },
    {
        "query": "Transform 'world'",
        "correct_answer": "Orldway"
    },
    {
        "query": "Transform 'python'",
        "correct_answer": "Ythonpay"
    },
    {
        "query": "Transform 'code'",
        "correct_answer": "Odecay"
    },
    {
        "query": "Transform 'learn'",
        "correct_answer": "Earnlay"
    }
]

def train_pattern_learner():
    memory = Memory("mem_aa.json")
    # memory.positive_knowledge = []
    # memory.negative_knowledge = []
    # memory.next_knowledge_id = 1
    memory._save()

    for epoch in range(5):
        print(f"Epoch {epoch + 1}")
        for i, example in enumerate(training_examples, 1):
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation pattern that converts the input to output",
                memory=memory
            )
            # print(f"Query: {example['query']}")
            print(f"System Answer: {result['answer']} | Correct Answer: {example['correct_answer']}")
            # print(f"Feedback: {result['feedback']}")
            print("-" * 50)
        print("=" * 50)
train_pattern_learner()

Epoch 1
System Answer: ellohay | Correct Answer: Ellohay
--------------------------------------------------
System Answer: orldway | Correct Answer: Orldway
--------------------------------------------------
System Answer: onpythay | Correct Answer: Ythonpay
--------------------------------------------------
System Answer: odecay | Correct Answer: Odecay
--------------------------------------------------
System Answer: earnlay | Correct Answer: Earnlay
--------------------------------------------------
Epoch 2
System Answer: ellohay | Correct Answer: Ellohay
--------------------------------------------------
System Answer: orldway | Correct Answer: Orldway
--------------------------------------------------
System Answer: onpythay | Correct Answer: Ythonpay
--------------------------------------------------
System Answer: odecay | Correct Answer: Odecay
--------------------------------------------------
System Answer: earnlway | Correct Answer: Earnlay
----------------------------------

In [165]:
from builtins import print


def transform_sentence(sentence,n=2):
    n-=1
    words = sentence.split()
    transformed = ''.join(word[n] for word in words if len(word) >=n)
    return transformed


# Training and testing data generation
sentences = [
    "python is fun",
    "code is life",
    "learn and grow",
    "practice makes perfect",
    "knowledge is power",
    "ai is the future",
    "data drives decisions",
    "science is amazing",
    "always keep learning",
    "perseverance pays off",
    "learning is ongoing",
    "practice brings mastery",
    "knowledge fuels creativity",
    "transform ideas daily",
    "explore and create",
    "analyze your potential",
    "never stop growing",
    "think outside boundaries",
    "focus on improvement",
    "believe in progress"
]

nth_word=1
# Generate examples
examples = [
    {
        "query": f"Transform '{sentence}'",
        "correct_answer": transform_sentence(sentence,n=nth_word)
    }
    for sentence in sentences
]

# Split examples into training and testing (10 each)
training_examples = examples[:10]
testing_examples = examples[10:]


# Modified train_pattern_learner function
def train_pattern_learner():
    memory = Memory("mem3.json")
    memory.positive_knowledge = []
    memory.negative_knowledge = []
    memory.next_knowledge_id = 1
    memory._save()

    for epoch in range(5):
        print(f"Epoch {epoch + 1}")
        for i, example in enumerate(training_examples, 1):
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation rule that converts the input to output and return the correct output",
                memory=memory
            )
            
            print(f"System Answer: {result.get('answer', 'No Answer')} | Correct Answer: {example['correct_answer']}")
        
        print("-" * 50)

train_pattern_learner()

Epoch 1
System Answer: Python is enjoyable. | Correct Answer: pif
System Answer: CIL | Correct Answer: cil
System Answer: L&G | Correct Answer: lag
System Answer: pmp | Correct Answer: pmp
System Answer: k.i.p. | Correct Answer: kip
System Answer: ai | Correct Answer: aitf
System Answer: dd | Correct Answer: ddd
System Answer: sim | Correct Answer: sia
System Answer: AKL | Correct Answer: akl
System Answer: PPO | Correct Answer: ppo
--------------------------------------------------
Epoch 2
System Answer: PIF | Correct Answer: pif
System Answer: CIL | Correct Answer: cil
System Answer: lag | Correct Answer: lag
System Answer: PM | Correct Answer: pmp
System Answer: KIP | Correct Answer: kip
System Answer: ai-f-t | Correct Answer: aitf
System Answer: dd | Correct Answer: ddd
System Answer: siA | Correct Answer: sia
System Answer: akl | Correct Answer: akl
System Answer: PPO | Correct Answer: ppo
--------------------------------------------------
Epoch 3
System Answer: PIF | Correct Answ

### M shot example

In [181]:
from builtins import print
import random

def transform_sentence(sentence, n=2):
    n -= 1
    words = sentence.split()
    transformed = ''.join(word[n] for word in words if len(word) > n)
    return transformed

# Training and testing data generation
sentences = [
    "python is fun",
    "code is life",
    "learn and grow",
    "practice makes perfect",
    "knowledge is power",
    "ai is the future",
    "data drives decisions",
    "science is amazing",
    "always keep learning",
    "perseverance pays off",
    "learning is ongoing",
    "practice brings mastery",
    "knowledge fuels creativity",
    "transform ideas daily",
    "explore and create",
    "analyze your potential",
    "never stop growing",
    "think outside boundaries",
    "focus on improvement",
    "believe in progress"
]

nth_word = 1
m_shot = 0  # Number of shot examples

# Generate examples with m-shot context
examples = []

for i, sentence in enumerate(sentences):
    shot_examples = random.sample([s for s in sentences if s != sentence], min(m_shot, len(sentences) - 1))  # Select random m-shot examples excluding the current sentence
    shots = "\n".join(
        [f"If \"{s}\" is transformed to \"{transform_sentence(s, n=nth_word)}\"" for s in shot_examples]
    )

    query = f"{shots}\nthen what is \"{sentence}\" transformed to?" if shots else f"What is \"{sentence}\" transformed to?"
    examples.append(
        {
            "query": query,
            "correct_answer": transform_sentence(sentence, n=nth_word)
        }
    )

# Split examples into training and testing (10 each)
training_examples = examples[:10]
testing_examples = examples[10:]

# Modified train_pattern_learner function
def train_pattern_learner():
    memory = Memory("mem_0_shot_1_shift.json")
    memory.positive_knowledge = []
    memory.negative_knowledge = []
    memory.next_knowledge_id = 1
    memory._save()

    for epoch in range(5):
        print(f"Epoch {epoch + 1}")
        for i, example in enumerate(training_examples, 1):
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation rule that converts the input to output and return the correct output",
                memory=memory
            )

            print(f"System Answer: {result.get('answer', 'No Answer')} | Correct Answer: {example['correct_answer']}")

        print("-" * 50)

train_pattern_learner()


Epoch 1
System Answer: Python is enjoyable | Correct Answer: pif
System Answer: "code is life" can be transformed to "life is code". | Correct Answer: cil
System Answer: L&G | Correct Answer: lag
System Answer: Skill development through repetition | Correct Answer: pmp
System Answer: KIP | Correct Answer: kip
System Answer: AI-Future | Correct Answer: aitf
System Answer: D3 | Correct Answer: ddd
System Answer: SIA (Science Is Amazing) | Correct Answer: sia
System Answer: AKL | Correct Answer: akl
System Answer: PPOP | Correct Answer: ppo
--------------------------------------------------
Epoch 2
System Answer: PYTHON = Fun | Correct Answer: pif
System Answer: CIL | Correct Answer: cil
System Answer: L&G | Correct Answer: lag
System Answer: PMP (Practice Makes Perfect) | Correct Answer: pmp
System Answer: KIP | Correct Answer: kip
System Answer: AI is the future. | Correct Answer: aitf
System Answer: D3 | Correct Answer: ddd
System Answer: SIA (Science Is Amazing) | Correct Answer: sia
